
# Intel® Movidius™ Neural Compute Stick (NCS)

This lab shows how the Intel® Distribution of OpenVINO™ toolkit provides hardware abstraction to run the sample object detection application which was built in previous modules on Intel® Movidius™ Neural Compute Stick.

###  Importing dependencies, Setting the Environment variables, downloading models and Generate the IR files

In [ ]:
from IPython.display import HTML
import os
import time
import sys                                     
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent.parent))
from demoTools.demoutils import *
import matplotlib.pyplot as plt

In [ ]:
! /opt/intel/openvino/bin/setupvars.sh

In [ ]:
!ln -sf /data/reference-sample-data/object-detection-python/cars_1900.mp4 
videoHTML('Cars video', ['cars_1900.mp4'])

In [ ]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name mobilenet-ssd -o models

In [ ]:
! cd models/public/mobilenet-ssd/  && mkdir -p FP16 && mkdir -p FP32

In [ ]:
!python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_caffe.py --input_model models/public/mobilenet-ssd/mobilenet-ssd.caffemodel -o models/mobilenet-ssd/FP32 --scale 256 --mean_values [127,127,127] 

The Model Optimizer by default generate FP32 IR files if the data type is not particularly specified.
Let's run the Model Optimizer to get IR files in FP16 format suitable for the Intel® Movidius™ NCS by setting the data_type flag to FP16

In [ ]:
!python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_caffe.py --input_model models/public/mobilenet-ssd/mobilenet-ssd.caffemodel -o models/mobilenet-ssd/FP16/ --scale 256 --mean_values [127,127,127] --data_type FP16

In [ ]:
! make

In [ ]:
 !cd models/mobilenet-ssd/FP16 && ls

Now run the example application with these new IR files.


    
## Run the sample application on Intel® Movidius™ Neural Compute Stick (NCS)

#### Create Job Script 

We will run the workload on several DevCloud's edge compute nodes. We will send work to the edge compute nodes by submitting jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

The job file will be executed directly on the edge compute node. In this exercise, we will MYRIAD as the hardware

In [ ]:
%%writefile object_detection_job.sh

# The default path for the job is your home directory, so we change directory to where the files are.
cd $PBS_O_WORKDIR
OUTPUT_FILE=$1
DEVICE=$2
MODEL=$3
# Object detection script writes output to a file inside a directory. We make sure that this directory exists.
#  The output directory is the first argument of the bash script
mkdir -p $OUTPUT_FILE
ROIFILE=$OUTPUT_FILE/ROIs.txt
OVIDEO=$OUTPUT_FILE/output.mp4

if [ "$MODEL" = "FP32" ]; then
    config_file="conf_fp32.txt"
else
    config_file="conf_fp16.txt"
fi

# Running the object detection code
SAMPLEPATH=$PBS_O_WORKDIR
./tutorial1 -i /data/reference-sample-data/object-detection-python/cars_1900.mp4 \
            -m models/$MODEL \
            -d $DEVICE \
            -o $OUTPUT_FILE\
            -fr 3000 

# Converting the text output to a video
./ROI_writer -i cars_1900.mp4 \
             -o $OUTPUT_FILE \
             -ROIfile $ROIFILE \
             -l pascal_voc_classes.txt \
             -r 2.0 # output in half res

#### Submitting to an edge compute node with Intel® Movidius™ Neural Compute Stick 2


The Model Optimizer by default generate FP32 IR files if the data type is not particularly specified. Technically, the Intel® Movidius™ devices support FP16 quantization only. The movidius plugin converts the FP32 quantized model to FP16 during runtime and does through any error even if you provide FP32 quantized model.

Let's run the Model Optimizer to get IR files in FP16 format to infer the model on the Intel® Movidius™ NCS by setting the data_type flag to FP16.

In [ ]:
#Submit job to the queue
job_id_ncs2 = !qsub object_detection_job.sh -l nodes=1:idc004nc2:intel-ncs2 -F "resultsCpp/ncs2 MYRIAD mobilenet-ssd/FP16/mobilenet-ssd.xml" -N obj_det_ncs2
print(job_id_ncs2[0]) 
#Progress indicators
if job_id_ncs2:
    progressIndicator('resultsCpp/ncs2', 'i_progress_'+job_id_ncs2[0]+'.txt', "Inference", 0, 100)
    progressIndicator('resultsCpp/ncs2', 'v_progress_'+job_id_ncs2[0]+'.txt', "Rendering", 0, 100)

In [ ]:
videoHTML('IEI Tank (Intel  Movidius)', 
          ['resultsCpp/ncs2/output.mp4'])


## Run the example on different hardware

#### Submitting to an edge compute node with CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel 
    Core i5-6500TE</a>. The inference workload will run on the CPU.


In [ ]:
#Submit job to the queue
job_id_core = !qsub object_detection_job.sh -l nodes=1:tank-870:i5-6500te -l nodes=1:tank-870:i5-6500te -F "resultsCpp/core CPU mobilenet-ssd/FP32/mobilenet-ssd.xml" -N obj_det_core
print(job_id_core[0]) 
#Progress indicators
if job_id_core:
    progressIndicator('resultsCpp/core', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)
    progressIndicator('resultsCpp/core', 'v_progress_'+job_id_core[0]+'.txt', "Rendering", 0, 100)

In [ ]:
videoHTML('IEI Tank (Intel  Core)', 
          ['resultsCpp/core/output.mp4'])

#### Submitting to a node with Intel® Core CPU and using the onboard Intel GPU

In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core i5-6500TE</a>. The inference workload will run on the Intel® HD Graphics 530 card integrated with the CPU.

Set target hardware as GPU with -d GPU

In [ ]:
#Submit job to the queue
job_id_gpu = !qsub object_detection_job.sh -l nodes=1:idc001skl:intel-hd-530 -F "resultsCpp/gpu GPU mobilenet-ssd/FP32/mobilenet-ssd.xml" -N obj_det_gpu
print(job_id_gpu[0]) 
#Progress indicators
if job_id_gpu:
    progressIndicator('resultsCpp/gpu', 'i_progress_'+job_id_gpu[0]+'.txt', "Inference", 0, 100)
    progressIndicator('resultsCpp/gpu', 'v_progress_'+job_id_gpu[0]+'.txt', "Rendering", 0, 100)

In [ ]:
videoHTML('IEI Tank (Intel  GPU)', 
          ['resultsCpp/gpu/output.mp4'])


## Assess Performance

The running time of each inference task is recorded in `results/*/stats.txt`, where the subdirectory name corresponds to the architecture of the target edge compute node. Run the cell below to plot the results of all jobs side-by-side. Lower values mean better performance. Keep in mind that some architectures are optimized for the highest performance, others for low power or other metrics.


In [ ]:
arch_list = [('core', 'Intel Core\ni5-6500TE\nCPU'),
             ('gpu', ' Intel Core\ni5-6500TE\nGPU'),
             ('ncs2', 'Intel\nNCS2')]
stats_list = []
for arch, a_name in arch_list:
    if 'job_id_'+arch in vars():
        stats_list.append(('resultsCpp/{arch}/stats.txt'.format(arch=arch), a_name))
    else:
        stats_list.append(('placeholder'+arch, a_name))
summaryPlot(stats_list, 'Architecture', 'Time, seconds', 'Inference Engine Processing Time', 'time', 0.4)
summaryPlot(stats_list, 'Architecture', 'Frames per second', 'Inference Engine FPS', 'fps', 0.4 )